In [1]:
import subprocess
import shlex
import re

import numpy as np

from IPython.display import display, Markdown
from tqdm import tqdm

In [2]:
CC_PATTERN = re.compile(r"(?<!^)(?=[A-Z])")

# PractRand (Practically Random) - statistical tests for RNGs

In [3]:
def test_randomness_practrand(
    mode, seeds=[42], limit='1GB',
):
    outputs = []
    
    for seed in tqdm(seeds):
        generator = subprocess.Popen(shlex.split(
            f"cargo run --release --quiet -- --seed {seed} {CC_PATTERN.sub('-', mode).lower()}"
        ), stdout=subprocess.PIPE)
        practrand = subprocess.run(
            shlex.split(f"./practrand stdin64 -tlmax {limit}"), stdin=generator.stdout,
            check=True, capture_output=True, text=True
        )

        outputs.append(practrand.stdout)
        
    most_suspect = None
    
    for output in outputs:
        failures = output.count("FAIL")
        suspicions = output.count("SUSPICIOUS") * 2 + output.count("suspicious")
        unusuals = output.count("unusual")
        
        suspicion = failures * 10 + suspicions * 2 + unusuals
        
        if (most_suspect is None) or (suspicion > most_suspect[0]):
            most_suspect = (suspicion, output)

    if "FAIL" in most_suspect[1]:
        display(Markdown(f"## <span style='color:purple'><u>{mode}</u></span>"))
    elif "SUSPICIOUS" in most_suspect[1]:
        display(Markdown(f"## <span style='color:red'><u>{mode}</u></span>"))
    elif "suspicious" in most_suspect[1]:
        display(Markdown(f"## <span style='color:orange'>*{mode}*</span>"))
    else:
        display(Markdown(f"## <span style='color:green'>{mode}</span>"))
        
    display(Markdown("#### Test output (most suspect):"))
    display(Markdown(">" + most_suspect[1].replace('\n', '\n>')))

    display(Markdown("#### Parameters:"))
    display(Markdown(f"* seeds: {seeds}\n* limit: {limit}"))

In [4]:
for mode in ["Monolithic", "Independent", "IndependentSimulation", "IndependentSimulationNoDispersal"]:
    seeds = np.random.randint(0, np.iinfo("uint64").max, dtype="uint64", size=10)
    
    test_randomness_practrand(mode, seeds=seeds, limit='512GB')

100%|██████████| 10/10 [18:50:56<00:00, 6785.70s/it]


## <span style='color:green'>Monolithic</span>

#### Test output (most suspect):

>RNG_test using PractRand version 0.95
>RNG = RNG_stdin64, seed = unknown
>test set = core, folding = standard (64 bit)
>
>rng=RNG_stdin64, seed=unknown
>length= 256 megabytes (2^28 bytes), time= 3.2 seconds
>  Test Name                         Raw       Processed     Evaluation
>  [Low4/64]mod3n(5):(1,9-6)         R= +10.7  p =  1.6e-4   unusual          
>  ...and 212 test result(s) without anomalies
>
>rng=RNG_stdin64, seed=unknown
>length= 512 megabytes (2^29 bytes), time= 7.9 seconds
>  no anomalies in 229 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 1 gigabyte (2^30 bytes), time= 15.9 seconds
>  no anomalies in 246 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 2 gigabytes (2^31 bytes), time= 30.7 seconds
>  no anomalies in 263 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 4 gigabytes (2^32 bytes), time= 58.5 seconds
>  no anomalies in 279 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 8 gigabytes (2^33 bytes), time= 115 seconds
>  Test Name                         Raw       Processed     Evaluation
>  [Low4/64]DC6-9x1Bytes-1           R=  -5.1  p =1-1.4e-3   unusual          
>  ...and 294 test result(s) without anomalies
>
>rng=RNG_stdin64, seed=unknown
>length= 16 gigabytes (2^34 bytes), time= 224 seconds
>  Test Name                         Raw       Processed     Evaluation
>  Gap-16:B                          R=  -4.6  p =1-7.7e-4   unusual          
>  [Low4/64]DC6-9x1Bytes-1           R=  -5.0  p =1-1.7e-3   unusual          
>  ...and 309 test result(s) without anomalies
>
>rng=RNG_stdin64, seed=unknown
>length= 32 gigabytes (2^35 bytes), time= 435 seconds
>  no anomalies in 325 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 64 gigabytes (2^36 bytes), time= 861 seconds
>  no anomalies in 340 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 128 gigabytes (2^37 bytes), time= 1714 seconds
>  no anomalies in 355 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 256 gigabytes (2^38 bytes), time= 3392 seconds
>  no anomalies in 369 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 512 gigabytes (2^39 bytes), time= 6809 seconds
>  no anomalies in 383 test result(s)
>
>

#### Parameters:

...

## <span style='color:green'>IndependentSimulationNoDispersal</span>

#### Test output (most suspect):

>RNG_test using PractRand version 0.95
>RNG = RNG_stdin64, seed = unknown
>test set = core, folding = standard (64 bit)
>
>rng=RNG_stdin64, seed=unknown
>length= 128 megabytes (2^27 bytes), time= 2.1 seconds
>  no anomalies in 196 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 256 megabytes (2^28 bytes), time= 5.5 seconds
>  no anomalies in 213 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 512 megabytes (2^29 bytes), time= 11.2 seconds
>  no anomalies in 229 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 1 gigabyte (2^30 bytes), time= 20.6 seconds
>  Test Name                         Raw       Processed     Evaluation
>  [Low1/64]DC6-9x1Bytes-1           R=  +6.4  p =  2.2e-3   unusual          
>  [Low1/64]mod3n(5):(0,9-6)         R= +12.4  p =  4.4e-5   unusual          
>  ...and 244 test result(s) without anomalies
>
>rng=RNG_stdin64, seed=unknown
>length= 2 gigabytes (2^31 bytes), time= 37.8 seconds
>  Test Name                         Raw       Processed     Evaluation
>  [Low1/64]DC6-9x1Bytes-1           R=  +6.9  p =  7.2e-4   unusual          
>  ...and 262 test result(s) without anomalies
>
>rng=RNG_stdin64, seed=unknown
>length= 4 gigabytes (2^32 bytes), time= 70.5 seconds
>  Test Name                         Raw       Processed     Evaluation
>  [Low1/64]mod3n(5):(0,9-6)         R= +10.7  p =  1.6e-4   unusual          
>  ...and 278 test result(s) without anomalies
>
>rng=RNG_stdin64, seed=unknown
>length= 8 gigabytes (2^33 bytes), time= 135 seconds
>  no anomalies in 295 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 16 gigabytes (2^34 bytes), time= 263 seconds
>  no anomalies in 311 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 32 gigabytes (2^35 bytes), time= 513 seconds
>  no anomalies in 325 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 64 gigabytes (2^36 bytes), time= 1026 seconds
>  no anomalies in 340 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 128 gigabytes (2^37 bytes), time= 2037 seconds
>  no anomalies in 355 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 256 gigabytes (2^38 bytes), time= 4069 seconds
>  no anomalies in 369 test result(s)
>
>rng=RNG_stdin64, seed=unknown
>length= 512 gigabytes (2^39 bytes), time= 8140 seconds
>  Test Name                         Raw       Processed     Evaluation
>  [Low16/64]FPF-14+6/16:cross       R=  +4.9  p =  1.8e-4   unusual          
>  ...and 382 test result(s) without anomalies
>
>

#### Parameters:

* seeds: [16448020747669778562  4368962271258711975 11249100981175841188
  2365975233776638442   648488755577100847 16357435805637215904
  1895196650840720663  4210738645265698800 12509924760740056186
  1447442073382594683]
* limit: 512GB